# CONTROLLO MATCHING

In [18]:
import os
import pandas as pd
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma, DistributionBased, Cupid
import pprint
import numpy as np

def list2string(x):
    result = ''
    if(isinstance(x, list)):
        for element in x:
            result = result + ', ' + element
            return result[1:]
    else:
        return x

# Load data using pandas
df1 = pd.read_json("schema_mediato_confronto.json")
# df2 = pd.read_json("datasets/output_globaldata.json")
df2 = pd.read_json("datasets/valueToday_dataset.jsonl", lines=True)
df2 = df2.apply(np.vectorize(list2string))

# Instantiate matcher and run
# Coma requires java to be installed on your machine
# If java is not an option, all the other algorithms are in Python (e.g., Cupid)
# matcher = Coma(strategy="COMA_OPT")
matcher = DistributionBased(0.9, 0.9)
matches = valentine_match(df1, df2, matcher)
# print(type(matches))
# print(matches)
pp = pprint.PrettyPrinter(indent=4)

In [19]:
pp.pprint(matches)

{(('table_1', 'headquarters'), ('table_2', 'headquarters_region_city')): 0.918299407988494, (('table_1', 'website'), ('table_2', 'company_website')): 0.9141177743412015, (('table_1', 'name'), ('table_2', 'name')): 0.8835215044000037, (('table_1', 'country'), ('table_2', 'headquarters_country')): 0.8697265607735573, (('table_1', 'sector'), ('table_2', 'company_business')): 0.824344321652341, (('table_1', 'area_served'), ('table_2', 'headquarters_continent')): 0.7433102081268582, (('table_1', 'employees'), ('table_2', 'world_rank')): 0.7027325996082138, (('table_1', 'employees'), ('table_2', 'number_of_employees')): 0.7022677228450745}


# CREAZIONE SCHEMA MEDIATO

In [4]:
import os
import pandas as pd
from valentine import valentine_match, valentine_metrics
from valentine.algorithms import Coma, DistributionBased
import pprint

In [5]:
# ---------------------------------------------------------
# CREATA PER INIZIALIZZARE UN DATASET FITTIZIO MA ORA NON
# SI UTILIZZA IL DATASET FITTIZIO DA NOI CREATO MANUALMENTE
# ---------------------------------------------------------
def creation_dataset_confronto(df1):
  # Load data using pandas
  df2 = pd.read_json("datasets/output_disfold.json")

  matcher_coma = Coma(strategy="COMA_OPT")
  matches_coma = valentine_match(df1, df2, matcher_coma)

  attributi_mediato = []
  attributi_dataset = []
  for i in matches_coma:
    if matches_coma[i] > 0.5:
      attributi_mediato.append(i[0][1])
      attributi_dataset.append(i[1][1])

  for i in df2.index:
      data = {}
      # print(i)
      for d,m in zip(attributi_dataset,attributi_mediato):
        # print(df2[d][i])
        data[m] = df2[d][i]
      temp_data=pd.DataFrame([data])
      df1=pd.concat([df1, temp_data], ignore_index=True)

  return df1


In [11]:
def creation_df_log_matcher(df_logmachers, matches, i, type, name_dataset):
  record = {}
  record['tab1'] = i[0][0]
  record['attr1'] = i[0][1]
  record['tab2'] = name_dataset
  record['attr2'] = i[1][1]
  record['prob'] = matches[i]
  if type == 'coma':
    record['matcher'] = 'Coma'
  else:
    record['matcher'] = 'DistributionBased'
  temp_record=pd.DataFrame([record])
  # df_logmachers=df_logmachers.append(record, ignore_index=True)
  df_logmachers=pd.concat([df_logmachers, temp_record], ignore_index=True)
  return df_logmachers

In [12]:
def creation_schema_alignment(df1, df2, df_confronto, df_logmachers, name_dataset):

  # Instantiate matcher and run
  # Coma requires java to be installed on your machine
  # If java is not an option, all the other algorithms are in Python (e.g., Cupid)
  matcher_coma = Coma(strategy="COMA_OPT")
  matches_coma = valentine_match(df1, df2, matcher_coma)

  distr_run = False

 
  attributi_mediato = []
  attributi_dataset = []
  for i in matches_coma:
    df_logmachers = creation_df_log_matcher(df_logmachers, matches_coma, i, 'coma', name_dataset)
    if matches_coma[i] > 0.5:
      attributi_mediato.append(i[0][1])
      attributi_dataset.append(i[1][1])
    else:
      try:
        if not distr_run:
          matcher_distr = DistributionBased(0.9, 0.9)
          matches_distr = valentine_match(df_confronto, df2, matcher_distr)
          distr_run = True
        # distribution based già instanziato
        df_logmachers = creation_df_log_matcher(df_logmachers, matches_distr, i, 'distribution', name_dataset)
        if matches_distr[i] > 0.5:
          attributi_mediato.append(i[0][1])
          attributi_dataset.append(i[1][1])
      except:
        pass
  # print(attributi_dataset)
  # print(attributi_mediato)

  # inserimento dei dati nel dataframe
  for i in df2.index:
    data = {}
    # print(i)
    for d,m in zip(attributi_dataset,attributi_mediato):
      # print(df2[d][i])
      data[m] = df2[d][i]
    temp_data=pd.DataFrame([data])
    df1=pd.concat([df1, temp_data], ignore_index=True)

  pp = pprint.PrettyPrinter(indent=4)

  # stampa dei matchers
  pp.pprint(matches_coma)
  if distr_run:
    pp.pprint(matches_distr)

  return df1, df_logmachers
  # print(df1)


In [13]:
from pathlib import Path
#formati json, csv, xls, xlsx, jsonl
df1 = pd.DataFrame(columns = ['name' , 'industry', 'market_cap', 'employees', 'country', 'headquarters', 'address', 'sector', 'ceo', 'founded', 'share_price', 'website', 'stock', 'area_served', 'earnings'])
# df_confronto = creation_dataset_confronto(df1)
df_confronto = pd.read_json('schema_mediato_confronto.json')
df_logmachers = pd.DataFrame(columns = ['tab1' , 'attr1', 'tab2', 'attr2', 'prob', 'matcher'])

#numero di errori nell'esecuzione di file
num_error = 0

directory = 'datasets'
dataset_list_json = Path(directory).glob('*.json')
dataset_list_jsonl = Path(directory).glob('*.jsonl')
dataset_list_csv = Path(directory).glob('*.csv')
dataset_list_xls = Path(directory).glob('*.xls')
dataset_list_xlsx = Path(directory).glob('*.xlsx')
i = 0

# --------------------------------------------------------------------
# Modificare seleziona_tipo per scegliere che tipo di file eseguire
# --------------------------------------------------------------------
seleziona_tipo = 2

# CON SELEZIONA_TIPO=0 SI RUNNANO I TUTTI TIPI DI FILE
# CON SELEZIONA_TIPO=1 SI RUNNANO I JSON
if seleziona_tipo==0 or seleziona_tipo==1:
  for path in dataset_list_json:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_json(path)
    name_dataset = str(path).split('\\')[1]
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers, name_dataset)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=2 SI RUNNANO I JSONL
if seleziona_tipo==0 or seleziona_tipo==2:
  for path in dataset_list_jsonl:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_json(path, lines=True)
    name_dataset = str(path).split('\\')[1]
    print(name_dataset)
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers, name_dataset)
    except Exception as e:
      print(e)
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=3 SI RUNNANO I CSV
if seleziona_tipo==0 or seleziona_tipo==3:
  for path in dataset_list_csv:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_csv(path)
    name_dataset = str(path).split('\\')[1]
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers, name_dataset)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=4 SI RUNNANO I XLS
if seleziona_tipo==0 or seleziona_tipo==4:
  for path in dataset_list_xls:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_excel(path)
    name_dataset = str(path).split('\\')[1]
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers, name_dataset)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


# CON SELEZIONA_TIPO=5 SI RUNNANO I XLSX
if seleziona_tipo==0 or seleziona_tipo==5:
  for path in dataset_list_xlsx:
    print(str(i) + " " + str(path))
    i = i+1
    df2 = pd.read_excel(path)
    name_dataset = str(path).split('\\')[1]
    try:
      df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers, name_dataset)
    except:
      print("ERRORE " + str(path))
      num_error = num_error + 1


print("Numero errori: " + str(num_error))
display(df1)
print('--------DATAFRAME LOG MATCHERS--------')
display(df_logmachers)

0 datasets\companiesMarketCap_dataset.jsonl
companiesMarketCap_dataset.jsonl
unhashable type: 'list'
ERRORE datasets\companiesMarketCap_dataset.jsonl
1 datasets\disfold_dataset.jsonl
disfold_dataset.jsonl
{   (('table_1', 'ceo'), ('table_2', 'ceo')): 0.8310757,
    (('table_1', 'employees'), ('table_2', 'employees')): 0.84929436,
    (('table_1', 'founded'), ('table_2', 'founded')): 0.8446504,
    (('table_1', 'headquarters'), ('table_2', 'headquarters_country')): 0.6355629,
    (('table_1', 'market_cap'), ('table_2', 'market_cap')): 0.851268,
    (('table_1', 'name'), ('table_2', 'name')): 0.83519655}
2 datasets\hitHorizons_dataset.jsonl
hitHorizons_dataset.jsonl
{   (('table_1', 'address'), ('table_2', 'address')): 0.797828,
    (('table_1', 'earnings'), ('table_2', 'sic_code')): 0.275279,
    (('table_1', 'founded'), ('table_2', 'id')): 0.29596865,
    (('table_1', 'industry'), ('table_2', 'industry')): 0.80027896,
    (('table_1', 'name'), ('table_2', 'name')): 0.7883742}
{}
3 data

,name,industry,market_cap,employees,country,headquarters,address,sector,ceo,founded,share_price,website,stock,area_served,earnings
0,Apple,NaN,$2.825 Trillion,"100,000",NaN,Cupertino,NaN,NaN,Mr. Timothy D. Cook,"April 1, 1976",NaN,NaN,NaN,NaN,NaN
1,Danaher,NaN,$208.03 Billion,"78,000",NaN,Washington,NaN,NaN,Mr. Rainer M. Blair,1969,NaN,NaN,NaN,NaN,NaN
2,Merck,NaN,$210.38 Billion,"67,000",NaN,Kenilworth,NaN,NaN,Mr. Robert M. Davis J.D.,1891,NaN,NaN,NaN,NaN,NaN
3,Bhp,NaN,A$281.66 Billion,"40,110",NaN,Melbourne,NaN,NaN,"Mr. Mike P. Henry B.Sc., BSc (Chem)",1885,NaN,NaN,NaN,NaN,NaN
4,Nike,NaN,$213.60 Billion,"73,300",NaN,Beaverton,NaN,NaN,Mr. John J. Donahoe II,"January 25, 1964",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,Legacy Srl,"Finance, Insurance, and Real Estate",NaN,NaN,NaN,NaN,"VIA DELLA ZECCA 2, BOLOGNA, 40121, BOLOGNA, ITALY",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1663,Cribis Holding Srl,"Finance, Insurance, and Real Estate",NaN,NaN,NaN,NaN,"VIA DELLA ZECCA 2, BOLOGNA, 40121, BOLOGNA, ITALY",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1664,Porsche Italia Spa,Retail Trade,NaN,NaN,NaN,NaN,"CORSO STATI UNITI 35, PADOVA, 35127, PADOVA, I...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1665,Intesa Sanpaolo Rbm Salute Spa,"Finance, Insurance, and Real Estate",NaN,NaN,NaN,NaN,"VIALE STELVIO 55/57, MILANO, 20159, MILANO, ITALY",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


--------DATAFRAME LOG MATCHERS--------


,tab1,attr1,tab2,attr2,prob,matcher
0,table_1,market_cap,disfold_dataset.jsonl,market_cap,0.851268,Coma
1,table_1,employees,disfold_dataset.jsonl,employees,0.849294,Coma
2,table_1,founded,disfold_dataset.jsonl,founded,0.844650,Coma
3,table_1,name,disfold_dataset.jsonl,name,0.835197,Coma
4,table_1,ceo,disfold_dataset.jsonl,ceo,0.831076,Coma
5,table_1,headquarters,disfold_dataset.jsonl,headquarters_country,0.635563,Coma
6,table_1,industry,hitHorizons_dataset.jsonl,industry,0.800279,Coma
7,table_1,address,hitHorizons_dataset.jsonl,address,0.797828,Coma
8,table_1,name,hitHorizons_dataset.jsonl,name,0.788374,Coma
9,table_1,founded,hitHorizons_dataset.jsonl,id,0.295969,Coma


In [51]:
df1.to_csv('schema_mediato.csv')

In [52]:
df_logmachers.to_csv('log_matchers.csv')

# Il codice sottostante è utilizzato per generare il dataframe di un singolo dataset

In [15]:
df1 = pd.DataFrame(columns = ['name' , 'industry', 'market_cap', 'employees', 'country', 'headquarters', 'address', 'sector', 'ceo', 'founded', 'share_price', 'website', 'stock', 'area_served', 'revenue'])
df_confronto = pd.read_json('schema_mediato_confronto.json')
df_logmachers = pd.DataFrame(columns = ['tab1' , 'attr1', 'tab2', 'attr2', 'prob', 'matcher'])


# df2 = pd.read_json('datasets/output_globaldata.json')
df2 = pd.read_json('prova.json')
df1, df_logmachers = creation_schema_alignment(df1, df2, df_confronto, df_logmachers, 'prova.json')

df1
df_logmachers

TypeError: unhashable type: 'list'